# Predicción del Precio del Polipropileno con Deep Learning (CNN 1D y LSTM)

Este notebook desarrolla un flujo completo para:

- Obtener datos históricos del precio del **polipropileno (PP)** desde la web (web scraping).
- Explorar y preprocesar la serie de tiempo.
- Construir modelos de **redes neuronales profundas** (LSTM y CNN 1D) para predecir el precio futuro.
- Evaluar el desempeño (MSE, MAPE) y generar **predicciones a futuro** (5–7 días).
- Preparar una tabla de predicciones para fechas específicas (24–28 de noviembre de 2025).


## 1. Introducción

### ¿Qué es el polipropileno?

El polipropileno (PP) es un **polímero termoplástico** muy usado en empaques, textiles, automotriz, construcción, electrodomésticos y muchos otros sectores. Es ligero, resistente a químicos y al calor, y relativamente barato, por eso es de los plásticos más consumidos a nivel mundial.

### ¿Por qué es relevante su precio?

El precio del PP impacta:

- **Costos de producción** de empaques, partes automotrices y bienes de consumo.
- **Márgenes** de fabricantes y transformadores de plástico.
- **Decisiones de compra y planeación** en cadenas de suministro.

Por eso, poder **modelar y predecir** su precio ayuda a:

- Negociar contratos y compras de materia prima.
- Planear inventarios y producción.
- Evaluar riesgos y escenarios de mercado.

### Objetivo del notebook

Usaremos datos históricos diarios del precio del PP para:

1. Construir un dataset limpio con columnas `Date` y `Price`.
2. Analizar el comportamiento del precio en el tiempo.
3. Entrenar modelos **LSTM** y **CNN 1D** para predicción un paso adelante.
4. Evaluar los modelos con **MSE y MAPE** en un conjunto de prueba.
5. Generar predicciones para los próximos días y preparar una tabla con las fechas
   del **24 al 28 de noviembre de 2025**.


## 2. Imports y configuración global

In [241]:
# Chunk 1

import os
import random

# ---- CONFIG GLOBAL DE REPRODUCIBILIDAD (ANTES de importar TF) ----
os.environ["PYTHONHASHSEED"] = "36"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Opcional: para máxima reproducibilidad, fuerza CPU:
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
   Dense, LSTM, Dropout, Conv1D, Flatten, Input
)
from tensorflow.keras.optimizers import Adam   

pio.renderers.default = "vscode"

# ---- Semillas ----
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# ---- Determinismo extra (si tu versión lo soporta) ----
try:
    tf.config.experimental.enable_op_determinism(True)
except Exception:
    pass

# Opcional: hilos (si TF aún no se ha inicializado)
try:
    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)
except RuntimeError as e:
    print("No pude cambiar hilos (TF ya estaba inicializado):", e)

# ---- Parámetros globales ----
START_DATE = '2021-01-11'  

WINDOW_SIZE = 5
TRAIN_FRACTION = 0.7
VAL_FRACTION = 0.15
TEST_FRACTION = 0.15  

N_FUTURE_DAYS = 5  

TARGET_DATES_5D = pd.to_datetime(
    ["2025-11-24", "2025-11-25", "2025-11-26", "2025-11-27", "2025-11-28"]
)


In [242]:
# from __future__ import annotations
# import typing
# from typing import Any
# import dataclasses
# import jax
# import jax.numpy as jnp
# import numpy as np
# import torch
# import IPython
# import treescope

## 3. Funciones auxiliares

In [243]:
# chunk 2


def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula MAPE en % evitando divisiones raras."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def create_windowed_dataset(series_1d, window_size):
    """
    Recibe un array 1D (serie escalada) y genera:
    X: [n_samples, window_size]
    y: [n_samples]
    """
    X, y = [], []
    for i in range(len(series_1d) - window_size):
        X.append(series_1d[i : i + window_size])
        y.append(series_1d[i + window_size])
    return np.array(X), np.array(y)


def forecast_future(model, last_window_scaled, scaler, n_future=5):
    """
    Hace forecast recursivo n_future pasos adelante.
    last_window_scaled: array shape (window_size,) ya escalado.
    Devuelve:
      future_scaled: array de tamaño n_future en escala normalizada
      future_inv: array en escala original
    """
    window = last_window_scaled.copy()
    preds_scaled = []

    for _ in range(n_future):
        x_input = window.reshape(1, -1, 1)  # (1, timesteps, features=1)
        pred_scaled = model.predict(x_input, verbose=0)[0, 0]
        preds_scaled.append(pred_scaled)
        # Desplazar ventana y agregar nuevo valor
        window = np.roll(window, -1)
        window[-1] = pred_scaled

    preds_scaled = np.array(preds_scaled)
    preds_inv = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    return preds_scaled, preds_inv


def plot_test_predictions(dates_test, y_test_inv, y_pred_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho solo en test."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_test_inv,
        mode="lines", name="Real (test)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_pred_inv,
        mode="lines", name="Predicho (test)"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Precio real vs predicho (set de prueba)",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_predictions(dates_all, y_all_inv, y_pred_all_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho para todo el histórico (train+val+test)."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (precio spot)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_pred_all_inv,
        mode="lines", name="Predicho"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Real vs predicción en todo el histórico",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_with_future(dates_all, y_all_inv, future_dates, future_inv, title_prefix="LSTM"):
    """Histórico completo + predicciones futuras."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (histórico)"
    ))
    fig.add_trace(go.Scatter(
        x=future_dates, y=future_inv,
        mode="lines+markers", name="Predicción futura"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Histórico completo + predicción de los próximos días",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()

## 4. Extraccion de la Data

In [244]:
#chunk 3

from alphacast import Alphacast
#%pip install alphacast
alphacast = Alphacast("ak_eX9r2t13iRJ6uDIbKzD1")

# df_raw = alphacast.datasets.dataset(40971).download_data("pandas")
# df_raw.to_parquet("polipropileno_data.parquet")
df_raw = pd.read_parquet("polipropileno_data.parquet")

df_raw = df_raw[df_raw['Industrial'] == 'Polypropylene']
df_raw["Date"] = pd.to_datetime(df_raw["Date"])
df_raw = df_raw.sort_values("Date")
df_raw = df_raw[["Date", "Price"]]
# df_raw = df_raw[df_raw["Date"] >= START_DATE]
df_raw = df_raw.reset_index(drop=True)
df = df_raw
df


,Date,Price
0,2013-02-28,10670.0
1,2014-03-03,10652.0
2,2014-03-04,10660.0
3,2014-03-05,10700.0
4,2014-03-06,10630.0
...,...,...
2746,2025-11-14,6446.0
2747,2025-11-17,6465.0
2748,2025-11-18,6427.0
2749,2025-11-19,6422.0


In [245]:
#chunk 4
fig = px.line(
    df,
    x="Date",
    y="Price",
    title="Precio del Polipropileno (Futuros) - Histórico (sin recorte)"
)
fig.update_xaxes(title="Fecha")
fig.update_yaxes(title="Precio (unidades del sitio fuente)")
fig.show()


## 5. Revisión del dataset

En esta sección:

- Revisamos la estructura del DataFrame (`info`, `describe`).
- Vemos el rango de fechas disponible.
- Agregamos un **corte opcional** a partir de una fecha (`START_DATE`) por si queremos trabajar
  solo con una ventana más reciente.
- Graficamos:
  - Precio vs tiempo (recortado).
  - Precio + media móvil de 30 días.

In [246]:
#chunk 5

print("Shape (filas, columnas):", df.shape)
print("\nInfo:")
print(df.info())

print("\nDescribe:")
display(df["Price"].describe())

print("\nRango de fechas disponible:")
print("Mínima:", df["Date"].min())
print("Máxima:", df["Date"].max())

# ---- Corte opcional por fecha ----
if START_DATE is not None:
    start_dt = pd.to_datetime(START_DATE)
    df = df[df["Date"] >= start_dt].copy()
    print(f"\nAplicando corte desde {start_dt.date()}:")
else:
    df = df.copy()
    print("\nSin corte de fecha, usando todo el histórico.")

print("Shape después del corte:", df.shape)
print("Nuevo rango de fechas:")
print("Mínima:", df["Date"].min())
print("Máxima:", df["Date"].max())


Shape (filas, columnas): (2751, 2)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2751 entries, 0 to 2750
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2751 non-null   datetime64[ns]
 1   Price   2751 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 43.1 KB
None

Describe:


count     2751.000000
mean      8354.924391
std       1182.589130
min       5874.000000
25%       7529.000000
50%       8271.000000
75%       8922.000000
max      12035.000000
Name: Price, dtype: float64


Rango de fechas disponible:
Mínima: 2013-02-28 00:00:00
Máxima: 2025-11-20 00:00:00

Aplicando corte desde 2021-01-11:
Shape después del corte: (1097, 2)
Nuevo rango de fechas:
Mínima: 2021-01-11 00:00:00
Máxima: 2025-11-20 00:00:00


In [247]:
#chunk 6

# Agregar media móvil de 30 días
df["MA_30"] = df["Price"].rolling(window=30, min_periods=1).mean()


# 1) Precio vs tiempo (recortado)
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["Price"],
    mode="lines", name="Precio diario"
))
fig.update_layout(
    title="Precio diario del PP (posible histórico recortado)",
    xaxis_title="Fecha",
    yaxis_title="Precio"
)
fig.show()

# 2) Precio + media móvil 30 días
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["Price"],
    mode="lines", name="Precio diario"
))
fig.add_trace(go.Scatter(
    x=df["Date"], y=df["MA_30"],
    mode="lines", name="Media móvil 30 días"
))
fig.update_layout(
    title="Precio del PP con media móvil de 30 días",
    xaxis_title="Fecha",
    yaxis_title="Precio"
)
fig.show()


## 6. Preprocesamiento

Pasos:

1. Normalizar la columna de precio (`MinMaxScaler`) para estabilizar el entrenamiento.
2. Crear un dataset supervisado usando ventanas deslizantes de tamaño `WINDOW_SIZE`:
   - Entrada: precios de los últimos `WINDOW_SIZE` días.
   - Salida: precio del día siguiente.
3. Dividir en **train / validation / test** respetando el orden temporal.
4. Darle forma a los tensores para que funcionen con LSTM y CNN 1D:
   - Shape: `(n_samples, timesteps, features)` con `features = 1`.


In [248]:
#chunk 7

# # 4.1 Normalización de la serie

# prices = df["Price"].values.reshape(-1, 1)

# scaler = MinMaxScaler(feature_range=(0, 1))
# prices_scaled = scaler.fit_transform(prices).flatten()

# # 4.2 Crear ventanas (X, y)

# X_all, y_all = create_windowed_dataset(prices_scaled, WINDOW_SIZE)

# # Fechas para cada y (la predicción se asocia al último día de la ventana)
# dates_all = df["Date"].values[WINDOW_SIZE:]

# print("Total muestras (ventanas):", X_all.shape[0])

# # 4.3 División temporal train / val / test

# n_samples = X_all.shape[0]
# train_end = int(n_samples * TRAIN_FRACTION)
# val_end = int(n_samples * (TRAIN_FRACTION + VAL_FRACTION))

# X_train = X_all[:train_end]
# y_train = y_all[:train_end]
# dates_train = dates_all[:train_end]

# X_val = X_all[train_end:val_end]
# y_val = y_all[train_end:val_end]
# dates_val = dates_all[train_end:val_end]

# X_test = X_all[val_end:]
# y_test = y_all[val_end:]
# dates_test = dates_all[val_end:]

# print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# # 4.4 Dar forma para modelos secuenciales (LSTM / CNN1D)
# # Añadimos dimensión de features (=1)

# X_train_seq = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_val_seq = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
# X_test_seq = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# input_shape = (WINDOW_SIZE, 1)
# print("Input shape para el modelo:", input_shape)


In [249]:
#chunk 8

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# =======================================
#  1) Split primero (sin escalar, sin ventanas)
# =======================================
feature_cols = ["Price", "MA_30", "ret1", "MA_7", "vol20"]
prices = df["Price"].values.reshape(-1, 1)   # <--- univariable OK
dates = df["Date"].values


N = len(prices)

train_end_idx = int(N * TRAIN_FRACTION)
val_end_idx   = int(N * (TRAIN_FRACTION + VAL_FRACTION))

prices_train_raw = prices[:train_end_idx]
prices_val_raw   = prices[train_end_idx:val_end_idx]
prices_test_raw  = prices[val_end_idx:]

dates_train_raw = dates[:train_end_idx]
dates_val_raw   = dates[train_end_idx:val_end_idx]
dates_test_raw  = dates[val_end_idx:]

print("Split bruto (sin ventanas):")
print("Train:", prices_train_raw.shape, "Val:", prices_val_raw.shape, "Test:", prices_test_raw.shape)

# =======================================
#  2) Crear ventanas por bloque (sin escalar)
# =======================================

def create_windowed_1d(series_1d, window_size):
    """
    series_1d: array 1D (sin escalar)
    return: X (num_samples, window_size), y (num_samples,)
    """
    X, y = [], []
    for i in range(len(series_1d) - window_size):
        X.append(series_1d[i:i+window_size])
        y.append(series_1d[i+window_size])
    return np.array(X), np.array(y)

# Ojo: aquí usamos el vector 1D (no reshape a (-1,1))
prices_train_vec = prices_train_raw.flatten()
prices_val_vec   = prices_val_raw.flatten()
prices_test_vec  = prices_test_raw.flatten()

X_train_raw_win, y_train_raw = create_windowed_1d(prices_train_vec, WINDOW_SIZE)
X_val_raw_win,   y_val_raw   = create_windowed_1d(prices_val_vec,   WINDOW_SIZE)
X_test_raw_win,  y_test_raw  = create_windowed_1d(prices_test_vec,  WINDOW_SIZE)

# Fechas asociadas a cada y (último día de la ventana dentro de cada bloque)
dates_train_win = dates_train_raw[WINDOW_SIZE:]
dates_val_win   = dates_val_raw[WINDOW_SIZE:]
dates_test_win  = dates_test_raw[WINDOW_SIZE:]

print("\nVentanas por bloque (sin escalar):")
print("Train:", X_train_raw_win.shape, "Val:", X_val_raw_win.shape, "Test:", X_test_raw_win.shape)

# =======================================
#  3) MinMaxScaler: fit en train, transform en val y test
# =======================================
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit(prices_train_raw)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(prices_train_raw)  

# Escalar cada bloque
prices_train_scaled_block = scaler.transform(prices_train_raw).flatten()
prices_val_scaled_block   = scaler.transform(prices_val_raw).flatten()
prices_test_scaled_block  = scaler.transform(prices_test_raw).flatten()
prices_scaled = np.concatenate([
    prices_train_scaled_block,
    prices_val_scaled_block,
    prices_test_scaled_block
])

# Recrear ventanas pero ahora sobre las versiones escaladas
X_train_scaled, y_train_scaled = create_windowed_1d(prices_train_scaled_block, WINDOW_SIZE)
X_val_scaled,   y_val_scaled   = create_windowed_1d(prices_val_scaled_block,   WINDOW_SIZE)
X_test_scaled,  y_test_scaled  = create_windowed_1d(prices_test_scaled_block,  WINDOW_SIZE)

print("\nVentanas escaladas por bloque:")
print("Train:", X_train_scaled.shape, "Val:", X_val_scaled.shape, "Test:", X_test_scaled.shape)

# =======================================
#  4) Dar forma (window_size, n_features=1) para LSTM/CNN
# =======================================

X_train_seq = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_val_seq   = X_val_scaled.reshape(X_val_scaled.shape[0],   X_val_scaled.shape[1],   1)
X_test_seq  = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1],  1)

y_train = y_train_scaled
y_val   = y_val_scaled
y_test  = y_test_scaled

dates_train = dates_train_win
dates_val   = dates_val_win
dates_test  = dates_test_win

input_shape = (WINDOW_SIZE, 1)
print("\nInput shape para el modelo (nuevo flujo):", input_shape)


Split bruto (sin ventanas):
Train: (767, 1) Val: (165, 1) Test: (165, 1)

Ventanas por bloque (sin escalar):
Train: (762, 5) Val: (160, 5) Test: (160, 5)

Ventanas escaladas por bloque:
Train: (762, 5) Val: (160, 5) Test: (160, 5)

Input shape para el modelo (nuevo flujo): (5, 1)


## 7. Arquitecturas del modelo (LSTM y CNN 1D)

A continuación se definen dos modelos base:

- **Modelo LSTM**:
  - Pensado para capturar dependencias temporales a largo plazo.

- **Modelo CNN 1D**:
  - Usa convoluciones 1D sobre la serie para aprender patrones locales.

### 7.1 Modelo LSTM (ejemplo sencillo)

En esta sección entrenamos un modelo LSTM base:

- Ventana de `WINDOW_SIZE` días de entrada.
- Dos capas LSTM de 64 unidades con Dropout.
- Función de pérdida: MSE.
- Métricas reportadas: MSE y MAPE en el conjunto de prueba.

Luego generamos:

1. Gráfica de **real vs predicho** en test.
2. Gráfica de **real vs predicho en todo el histórico** (train+val+test).
3. Gráfica de **histórico completo + predicción de los próximos `N_FUTURE_DAYS` días**.


In [250]:
#Chunk 9

input_shape = X_train_seq.shape[1:]  # (WINDOW_SIZE, 1)

# 1) Definición capa a capa
lstm_model = Sequential()

lstm_model.add(Input(shape=input_shape))

# LSTM 1: capta patrones a ~ventana completa
# lstm_model.add(
#     LSTM(
#         units=60,               
#         return_sequences=True,  # deja secuencia para la siguiente LSTM
#     )
# )

# # lstm_model.add(Dropout(0.15))   # regularización moderada

# LSTM 2: resume la secuencia en un estado final
lstm_model.add(
    LSTM(
        units=64,
        return_sequences=True
    )
)
lstm_model.add(
    LSTM(
        units=32,
        return_sequences=False
    )
)
# lstm_model.add(Dropout(0.15))
lstm_model.add(Dense(32, activation="relu"))
lstm_model.add(Dense(1)) # Salida: precio del día siguiente
from tensorflow.keras.losses import Huber

# 2) Compilación
lstm_model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='mse'
)
lstm_model.summary()

# 3) Entrenamiento
history_lstm = lstm_model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=200,
    batch_size=32,
    shuffle=False,      # <- esto es súper importante
    verbose=1
)


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 5, 64)          │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,401 (118.75 KB)

 Trainable params: 30,401 (118.75 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.9834 - val_loss: 0.8931
Epoch 2/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7091 - val_loss: 0.6656
Epoch 3/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3965 - val_loss: 0.3513
Epoch 4/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1522 - val_loss: 0.0738
Epoch 5/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0839 - val_loss: 0.0137
Epoch 6/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0839 - val_loss: 0.0135
Epoch 7/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0803 - val_loss: 0.0120
Epoch 8/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0774 - val_loss: 0.0122
Epoch 9/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0754 - val_loss: 0.0118
Epoch 10/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0733 - val_loss: 0.0116
Epoch 11/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0716 - val_loss: 0.0114
Epoch 12/200
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0

In [251]:
#chunk 10

# =========================
#     EVALUACIÓN TEST
# =========================

# Predicciones (escala normalizada)
y_test_pred_scaled_lstm = lstm_model.predict(X_test_seq).flatten()

# Invertir escala
y_test_inv_lstm = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_test_pred_inv_lstm = scaler.inverse_transform(
    y_test_pred_scaled_lstm.reshape(-1, 1)
).flatten()

# Métricas
mse_lstm = mean_squared_error(y_test_inv_lstm, y_test_pred_inv_lstm)
mape_lstm = mean_absolute_percentage_error(y_test_inv_lstm, y_test_pred_inv_lstm)

print(f"\n[ LSTM ] MSE (test):  {mse_lstm:.4f}")
print(f"[ LSTM ] MAPE (test): {mape_lstm:.2f}%")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

[ LSTM ] MSE (test):  1559.0780
[ LSTM ] MAPE (test): 0.41%


In [252]:
# chunk 11

# =========================
#       GRÁFICAS
# =========================

# 1) Real vs predicho solo en test
plot_test_predictions(
    dates_test,
    y_test_inv_lstm,
    y_test_pred_inv_lstm,
    title_prefix="LSTM"
)

# 2) Real vs predicho en todo el histórico (train + val + test)
y_all_pred_scaled_lstm = lstm_model.predict(
    np.concatenate([X_train_seq, X_val_seq, X_test_seq], axis=0)
).flatten()

y_all_inv_lstm = scaler.inverse_transform(
    np.concatenate([y_train, y_val, y_test]).reshape(-1, 1)
).flatten()
y_all_pred_inv_lstm = scaler.inverse_transform(
    y_all_pred_scaled_lstm.reshape(-1, 1)
).flatten()

dates_all_full_lstm = np.concatenate([dates_train, dates_val, dates_test])

plot_full_predictions(
    dates_all_full_lstm,
    y_all_inv_lstm,
    y_all_pred_inv_lstm,
    title_prefix="LSTM"
)

# 3) Histórico completo + predicción de los próximos N_FUTURE_DAYS días
last_window_scaled_lstm = prices_scaled[-WINDOW_SIZE:]
future_scaled_lstm, future_inv_lstm = forecast_future(
    lstm_model,
    last_window_scaled_lstm,
    scaler,
    n_future=N_FUTURE_DAYS
)

last_date = df["Date"].max()
future_dates_lstm = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=N_FUTURE_DAYS,
    freq="D"
)

plot_full_with_future(
    dates_all_full_lstm,
    y_all_inv_lstm,
    future_dates_lstm,
    future_inv_lstm,
    title_prefix="LSTM"
)

# Tabla de predicciones futuras (LSTM)
future_table_lstm = pd.DataFrame({
    "Fecha futura": future_dates_lstm,
    "Predicción del precio (LSTM)": future_inv_lstm
})
display(future_table_lstm)


34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,Fecha futura,Predicción del precio (LSTM)
0,2025-11-21,6383.895020
1,2025-11-22,6372.099609
2,2025-11-23,6359.632324
3,2025-11-24,6347.915039
4,2025-11-25,6336.620117


### 7.2 Modelo CNN 1D (ejemplo sencillo)

Ahora repetimos el flujo con una **red convolucional 1D**:

- Varias capas `Conv1D` para capturar patrones locales en la serie.
- Capa `Flatten` y `Dense` para producir la predicción.
- Igual que en LSTM, medimos MSE y MAPE en test y generamos 3 gráficas:
  1. Real vs predicho (test).
  2. Real vs predicho (todo el histórico).
  3. Histórico + predicción futura.


In [253]:
WINDOW_SIZE = 25 # en vez de 5
START_DATE = '2021-01-11'  


#SEED = 51

# os.environ["PYTHONHASHSEED"] = str(SEED)

# random.seed(SEED)
# np.random.seed(SEED)
# tf.random.set_seed(SEED)


In [254]:
# chunk 2


def mean_absolute_percentage_error(y_true, y_pred):
    """Calcula MAPE en % evitando divisiones raras."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def create_windowed_dataset(series_1d, window_size):
    """
    Recibe un array 1D (serie escalada) y genera:
    X: [n_samples, window_size]
    y: [n_samples]
    """
    X, y = [], []
    for i in range(len(series_1d) - window_size):
        X.append(series_1d[i : i + window_size])
        y.append(series_1d[i + window_size])
    return np.array(X), np.array(y)


def forecast_future(model, last_window_scaled, scaler, n_future=5):
    """
    Hace forecast recursivo n_future pasos adelante.
    last_window_scaled: array shape (window_size,) ya escalado.
    Devuelve:
      future_scaled: array de tamaño n_future en escala normalizada
      future_inv: array en escala original
    """
    window = last_window_scaled.copy()
    preds_scaled = []

    for _ in range(n_future):
        x_input = window.reshape(1, -1, 1)  # (1, timesteps, features=1)
        pred_scaled = model.predict(x_input, verbose=0)[0, 0]
        preds_scaled.append(pred_scaled)
        # Desplazar ventana y agregar nuevo valor
        window = np.roll(window, -1)
        window[-1] = pred_scaled

    preds_scaled = np.array(preds_scaled)
    preds_inv = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    return preds_scaled, preds_inv


def plot_test_predictions(dates_test, y_test_inv, y_pred_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho solo en test."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_test_inv,
        mode="lines", name="Real (test)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_test, y=y_pred_inv,
        mode="lines", name="Predicho (test)"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Precio real vs predicho (set de prueba)",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_predictions(dates_all, y_all_inv, y_pred_all_inv, title_prefix="LSTM"):
    """Gráfica real vs predicho para todo el histórico (train+val+test)."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (precio spot)"
    ))
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_pred_all_inv,
        mode="lines", name="Predicho"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Real vs predicción en todo el histórico",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()


def plot_full_with_future(dates_all, y_all_inv, future_dates, future_inv, title_prefix="LSTM"):
    """Histórico completo + predicciones futuras."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=dates_all, y=y_all_inv,
        mode="lines", name="Real (histórico)"
    ))
    fig.add_trace(go.Scatter(
        x=future_dates, y=future_inv,
        mode="lines+markers", name="Predicción futura"
    ))
    fig.update_layout(
        title=f"{title_prefix} - Histórico completo + predicción de los próximos días",
        xaxis_title="Fecha",
        yaxis_title="Precio"
    )
    fig.show()

In [255]:
#chunk 8

from sklearn.preprocessing import MinMaxScaler
import numpy as np

# =======================================
#  1) Split primero (sin escalar, sin ventanas)
# =======================================
feature_cols = ["Price", "MA_30", "ret1", "MA_7", "vol20"]
prices = df["Price"].values.reshape(-1, 1)   # <--- univariable OK
dates = df["Date"].values


N = len(prices)

train_end_idx = int(N * TRAIN_FRACTION)
val_end_idx   = int(N * (TRAIN_FRACTION + VAL_FRACTION))

prices_train_raw = prices[:train_end_idx]
prices_val_raw   = prices[train_end_idx:val_end_idx]
prices_test_raw  = prices[val_end_idx:]

dates_train_raw = dates[:train_end_idx]
dates_val_raw   = dates[train_end_idx:val_end_idx]
dates_test_raw  = dates[val_end_idx:]

print("Split bruto (sin ventanas):")
print("Train:", prices_train_raw.shape, "Val:", prices_val_raw.shape, "Test:", prices_test_raw.shape)

# =======================================
#  2) Crear ventanas por bloque (sin escalar)
# =======================================

def create_windowed_1d(series_1d, window_size):
    """
    series_1d: array 1D (sin escalar)
    return: X (num_samples, window_size), y (num_samples,)
    """
    X, y = [], []
    for i in range(len(series_1d) - window_size):
        X.append(series_1d[i:i+window_size])
        y.append(series_1d[i+window_size])
    return np.array(X), np.array(y)

# Ojo: aquí usamos el vector 1D (no reshape a (-1,1))
prices_train_vec = prices_train_raw.flatten()
prices_val_vec   = prices_val_raw.flatten()
prices_test_vec  = prices_test_raw.flatten()

X_train_raw_win, y_train_raw = create_windowed_1d(prices_train_vec, WINDOW_SIZE)
X_val_raw_win,   y_val_raw   = create_windowed_1d(prices_val_vec,   WINDOW_SIZE)
X_test_raw_win,  y_test_raw  = create_windowed_1d(prices_test_vec,  WINDOW_SIZE)

# Fechas asociadas a cada y (último día de la ventana dentro de cada bloque)
dates_train_win = dates_train_raw[WINDOW_SIZE:]
dates_val_win   = dates_val_raw[WINDOW_SIZE:]
dates_test_win  = dates_test_raw[WINDOW_SIZE:]

print("\nVentanas por bloque (sin escalar):")
print("Train:", X_train_raw_win.shape, "Val:", X_val_raw_win.shape, "Test:", X_test_raw_win.shape)

# =======================================
#  3) MinMaxScaler: fit en train, transform en val y test
# =======================================
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler.fit(prices_train_raw)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(prices_train_raw)  

# Escalar cada bloque
prices_train_scaled_block = scaler.transform(prices_train_raw).flatten()
prices_val_scaled_block   = scaler.transform(prices_val_raw).flatten()
prices_test_scaled_block  = scaler.transform(prices_test_raw).flatten()
prices_scaled = np.concatenate([
    prices_train_scaled_block,
    prices_val_scaled_block,
    prices_test_scaled_block
])

# Recrear ventanas pero ahora sobre las versiones escaladas
X_train_scaled, y_train_scaled = create_windowed_1d(prices_train_scaled_block, WINDOW_SIZE)
X_val_scaled,   y_val_scaled   = create_windowed_1d(prices_val_scaled_block,   WINDOW_SIZE)
X_test_scaled,  y_test_scaled  = create_windowed_1d(prices_test_scaled_block,  WINDOW_SIZE)

print("\nVentanas escaladas por bloque:")
print("Train:", X_train_scaled.shape, "Val:", X_val_scaled.shape, "Test:", X_test_scaled.shape)

# =======================================
#  4) Dar forma (window_size, n_features=1) para LSTM/CNN
# =======================================

X_train_seq = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_val_seq   = X_val_scaled.reshape(X_val_scaled.shape[0],   X_val_scaled.shape[1],   1)
X_test_seq  = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1],  1)

y_train = y_train_scaled
y_val   = y_val_scaled
y_test  = y_test_scaled

dates_train = dates_train_win
dates_val   = dates_val_win
dates_test  = dates_test_win

input_shape = (WINDOW_SIZE, 1)
print("\nInput shape para el modelo (nuevo flujo):", input_shape)


Split bruto (sin ventanas):
Train: (767, 1) Val: (165, 1) Test: (165, 1)

Ventanas por bloque (sin escalar):
Train: (742, 25) Val: (140, 25) Test: (140, 25)

Ventanas escaladas por bloque:
Train: (742, 25) Val: (140, 25) Test: (140, 25)

Input shape para el modelo (nuevo flujo): (25, 1)


In [256]:
#chunk 12
SEED = 63  # el mismo que usas arriba, o uno distinto si quieres “semilla CNN”

np.random.seed(SEED)
tf.random.set_seed(SEED)


input_shape = X_train_seq.shape[1:]  # (WINDOW_SIZE, 1)


# 1) Definición del modelo CNN 1D capa a capa
cnn_model = Sequential()
cnn_model.add(Input(shape=input_shape))


# Conv1D 1
cnn_model.add(
    Conv1D(
        filters=128,
        kernel_size=5,
        activation="relu",
        padding="same"      # <-- solución
    )
)
cnn_model.add(Dropout(0.1))

# Conv1D 2
cnn_model.add(
    Conv1D(
        filters=64,
        kernel_size=3,
        activation="relu",
        padding="same"       # <-- solución
    )
)
cnn_model.add(Dropout(0.1))


# Aplanar + densas finales
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation="relu"))
cnn_model.add(Dense(1))

# 2) Compilación
cnn_model.compile(
    optimizer=Adam(learning_rate=5e-4),
    loss="mse"
)

cnn_model.summary()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(
    monitor="val_loss",
    patience=20,
    restore_best_weights=True
)

history_cnn = cnn_model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=400,          # más épocas posibles
    batch_size=32,
    shuffle=False,       # mantenlo para reproducibilidad
    callbacks=[es],
    verbose=1
)



Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_38 (Conv1D)              │ (None, 25, 128)        │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_39 (Conv1D)              │ (None, 25, 64)         │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 230,465 (900.25 KB)

 Trainable params: 230,465 (900.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6173 - val_loss: 0.0270
Epoch 2/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3724 - val_loss: 0.1766
Epoch 3/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3336 - val_loss: 0.1993
Epoch 4/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1718 - val_loss: 0.0165
Epoch 5/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1616 - val_loss: 0.0129
Epoch 6/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1548 - val_loss: 0.0390
Epoch 7/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1416 - val_loss: 0.0194
Epoch 8/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1332 - val_loss: 0.0401
Epoch 9/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1153 - val_loss: 0.0179
Epoch 10/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1130 - val_loss: 0.0256
Epoch 11/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0913 - val_loss: 0.0084
Epoch 12/400
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0

In [257]:
#chunk 13

# =========================
#     EVALUACIÓN TEST
# =========================

y_test_pred_scaled_cnn = cnn_model.predict(X_test_seq).flatten()

y_test_inv_cnn = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_test_pred_inv_cnn = scaler.inverse_transform(
    y_test_pred_scaled_cnn.reshape(-1, 1)
).flatten()

mse_cnn = mean_squared_error(y_test_inv_cnn, y_test_pred_inv_cnn)
mape_cnn = mean_absolute_percentage_error(y_test_inv_cnn, y_test_pred_inv_cnn)

print(f"\n[ CNN1D ] MSE (test):  {mse_cnn:.4f}")
print(f"[ CNN1D ] MAPE (test): {mape_cnn:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 

[ CNN1D ] MSE (test):  1797.9290
[ CNN1D ] MAPE (test): 0.44%


In [258]:
#chunk 14

# =========================
#       GRÁFICAS
# =========================

# 1) Real vs predicho (test)
plot_test_predictions(
    dates_test,
    y_test_inv_cnn,
    y_test_pred_inv_cnn,
    title_prefix="CNN 1D"
)

# 2) Real vs predicho (train + val + test)
y_all_pred_scaled_cnn = cnn_model.predict(
    np.concatenate([X_train_seq, X_val_seq, X_test_seq], axis=0)
).flatten()

y_all_inv_cnn = scaler.inverse_transform(
    np.concatenate([y_train, y_val, y_test]).reshape(-1, 1)
).flatten()
y_all_pred_inv_cnn = scaler.inverse_transform(
    y_all_pred_scaled_cnn.reshape(-1, 1)
).flatten()

dates_all_full_cnn = np.concatenate([dates_train, dates_val, dates_test])

plot_full_predictions(
    dates_all_full_cnn,
    y_all_inv_cnn,
    y_all_pred_inv_cnn,
    title_prefix="CNN 1D"
)

# 3) Histórico completo + predicción futura
last_window_scaled_cnn = prices_scaled[-WINDOW_SIZE:]
future_scaled_cnn, future_inv_cnn = forecast_future(
    cnn_model,
    last_window_scaled_cnn,
    scaler,
    n_future=N_FUTURE_DAYS
)

last_date = df["Date"].max()
future_dates_cnn = pd.date_range(
    start=last_date + pd.Timedelta(days=1),
    periods=N_FUTURE_DAYS,
    freq="D"
)

plot_full_with_future(
    dates_all_full_cnn,
    y_all_inv_cnn,
    future_dates_cnn,
    future_inv_cnn,
    title_prefix="CNN 1D"
)

# Tabla de predicciones futuras (CNN 1D)
future_table_cnn = pd.DataFrame({
    "Fecha futura": future_dates_cnn,
    "Predicción del precio (CNN 1D)": future_inv_cnn
})
display(future_table_cnn)




32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


,Fecha futura,Predicción del precio (CNN 1D)
0,2025-11-21,6376.479004
1,2025-11-22,6358.171875
2,2025-11-23,6341.958984
3,2025-11-24,6327.694336
4,2025-11-25,6315.307129


## 8. Tabla de predicción para fechas específicas (24–28 Nov 2025)

La consigna del proyecto pide una tabla como:

Fecha futura | Predicción del precio ($/ton) | Comentarios

Aquí preparamos la estructura. Tú puedes reemplazar la columna de predicción con los
valores de tu **mejor modelo** (LSTM o CNN 1D), usando el forecast que generes.


In [259]:
# Tabla vacía con la estructura sugerida para el reporte final

tabla_5d = pd.DataFrame({
    "Fecha futura": TARGET_DATES_5D,
    "(lstm)Predicción del precio ($/ton)": [np.nan] * len(TARGET_DATES_5D),
    "(CNN)Predicción del precio ($/ton)": [np.nan] * len(TARGET_DATES_5D),
    "Comentarios": ["—"] * len(TARGET_DATES_5D)
})

tabla_5d


,Fecha futura,(lstm)Predicción del precio ($/ton),(CNN)Predicción del precio ($/ton),Comentarios
0,2025-11-24,NaN,NaN,—
1,2025-11-25,NaN,NaN,—
2,2025-11-26,NaN,NaN,—
3,2025-11-27,NaN,NaN,—
4,2025-11-28,NaN,NaN,—


In [263]:
N_TARGET = len(TARGET_DATES_5D)

# Predicciones con el LSTM
future_scaled_lstm, future_inv_lstm = forecast_future(
    lstm_model,
    last_window_scaled_lstm,
    scaler,
    n_future=N_TARGET
)

# Predicciones con la CNN
future_scaled_cnn, future_inv_cnn = forecast_future(
    cnn_model,
    last_window_scaled_cnn,
    scaler,
    n_future=N_TARGET
)

In [264]:
future_dates = pd.to_datetime(TARGET_DATES_5D)

pred_lstm_series = pd.Series(future_inv_lstm, index=future_dates)
pred_cnn_series = pd.Series(future_inv_cnn, index=future_dates)

In [265]:
tabla_5d = tabla_5d.copy()

tabla_5d["Fecha futura"] = pd.to_datetime(tabla_5d["Fecha futura"])

tabla_5d = tabla_5d.set_index("Fecha futura")

tabla_5d["(lstm)Predicción del precio ($/ton)"] = pred_lstm_series
tabla_5d["(CNN)Predicción del precio ($/ton)"] = pred_cnn_series

tabla_5d = tabla_5d.reset_index()

tabla_5d


,Fecha futura,(lstm)Predicción del precio ($/ton),(CNN)Predicción del precio ($/ton),Comentarios
0,2025-11-24,6383.895020,6376.479004,—
1,2025-11-25,6372.099609,6358.171875,—
2,2025-11-26,6359.632324,6341.958984,—
3,2025-11-27,6347.915039,6327.694336,—
4,2025-11-28,6336.620117,6315.307129,—
